In [32]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes wandb

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Access the Hugging Face token
hf_token = os.getenv("hf_token")

In [18]:

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

import peft

from peft import LoraConfig, PeftModel,  prepare_model_for_kbit_training
from trl import SFTTrainer

In [22]:
import pandas as pd
file_path = '../../cleaned/Sheger Press️.csv'

# Load the CSV file into a pandas DataFrame
dataset = pd.read_csv(file_path)

In [25]:
dataset.head()

,Unnamed: 0,telegeram_channel_id,telegram_channel_name,telegeram_channel_type,message_id,message_type,text,created_at,update_at,hashtags,emojis,symbols,links,mentions
0,1,1770199174,Sheger Press️️,public_channel,3,message,አንድ ላይ ተጋግዘን ትልቅ መወያያ መድረክ እንፍጠር ስለምታደርጉልኝ ድጋፍ...,2021-12-30T22:02:03,2023-05-01T18:34:46,[],,NaN,[],[]
1,2,1770199174,Sheger Press️️,public_channel,14,message,መረጃ የህወሃት ጦር ከመቀሌ ወደ አፋር አብኣላ የሚወስደውን መንገድ መዝጋ...,2022-05-03T14:31:44,2022-05-03T14:31:53,[],❗️❗️❗️ ...,NaN,[],[]
2,3,1770199174,Sheger Press️️,public_channel,15,message,አዲስ መረጃ የጠ/ሩ የምሳ ግብዣ እንዴት አለፈ ብፁኡ ወቅዱስ አቡነ ማትያ...,2022-05-03T15:25:59,2022-05-03T15:29:15,[],⁉️ ...,NaN,[],[]
3,4,1770199174,Sheger Press️️,public_channel,16,message,ልዩ መረጃ # የኦሮሚያ ልዩ ሃይል እና ሸኔ በምእራብ ሸዋ በአማሮች ላይ ...,2022-05-03T15:44:43,NaN,"['#', '#']",‼️ ...,-'',[],[]
4,5,1770199174,Sheger Press️️,public_channel,17,message,NEWSALERT ከቀናት በፊት ደሴ ላይ ቤተከርስቲያን አፍርሰን መስጊድ እ...,2022-05-03T17:07:43,2022-05-03T17:09:57,[],‼️,NaN,[],[]


In [7]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.2 MB/s eta 0:00:00a 0:00:01


In [8]:
from transformers import AutoTokenizer

# Model
base_model = "stabilityai/stablelm-2-1_6b"
new_model = "stableAI_Amh"



# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True, trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

arcade100k.tiktoken:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

In [14]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05, #the dropout probability of the LoRA layers
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)

In [19]:
model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [30]:
import pandas as pd
from trl.trainer import SFTTrainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments

In [29]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_steps=1,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    warmup_steps=10,
    report_to="wandb",
    # max_steps=2, # Remove this line for a real fine-tuning
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    peft_config=None,  # Make sure peft_config is initialized appropriately
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)


AttributeError: 'DataFrame' object has no attribute 'column_names'

In [ ]:

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)